# 环境的安装
```bash
conda create -n logic python==3.10
conda activate logic

pip install modelscope==1.9.5
pip install "transformers>=4.39.0"
pip install streamlit==1.24.0
pip install sentencepiece==0.1.99
pip install accelerate
pip install transformers_stream_generator==0.0.4
pip install datasets==2.18.0
pip install peft==0.10.0
pip install ipywidgets

<!-- pip install -U accelerate -->

lab add logic  刷新页面即可

pip install "transformers>=4.39.0"
pip install streamlit==1.24.0
pip install sentencepiece==0.1.99
pip install transformers_stream_generator==0.0.4
pip install datasets==2.18.0
pip install peft==0.10.0
pip install openai==1.17.1
pip install tqdm==4.64.1
pip install transformers==4.39.3
python -m pip install setuptools==69.5.1
pip install vllm==0.4.0.post1
pip install nest-asyncio
pip install accelerate
pip install tf-keras
pip install ipywidgets
```

# 导入环境

In [3]:
# mode_path = '/root/share/model_repos/internlm2-chat-7b'
mode_path = '/root/share/new_models/qwen/Qwen2-7B-Instruct'

output_lora_dir="./lora_model"
lora_path = f'{output_lora_dir}/checkpoint-1300'


In [2]:
from datasets import Dataset
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer, GenerationConfig

In [4]:
# 将JSON文件转换为CSV文件
df = pd.read_json('../data/QA.json')


In [5]:
ds = Dataset.from_pandas(df)

In [6]:
df

,question,answer
0,叶秋在被嘉世俱乐部驱逐后，如何重新找回自己的巅峰之路？,叶秋在被嘉世俱乐部驱逐后，通过在网游荣耀中创建账号‘一叶之秋’，并重新投入游戏，带着对往昔的...
1,叶修在俱乐部离开时，心情如何？,叶修在离开俱乐部时没有太多的话别，他一共只说了八个字：‘休息一年，然后回来。’苏沐橙站在俱乐...
2,叶修是如何被陈果发现并邀请成为网管的？,叶修被陈果发现并邀请成为网管的过程是这样的：陈果在一家网吧发现了一名玩家，这名玩家在网吧寻找...
3,叶修在陈果的网吧遇到了什么样的住宿条件？,叶修在陈果的网吧遇到了很差的住宿条件，小储物间里只有西墙顶上的一扇小窗，灯光昏暗，仿佛闹鬼。...
4,叶修是如何在40多秒内解决掉对手的？,叶修之所以能在40多秒内解决对手，是因为他是职业级的电竞选手。虽然他在陈果面前表现得像是新手...
...,...,...
2143,叶修为什么退役？,叶修因为嘉世战队的商业原因和管理层的决定，被迫退役。但他没有放弃自己的电竞梦想，转而在网吧当...
2144,君莫笑是什么角色？,君莫笑是叶修在《全职高手》中的新账号角色，这个账号是一个散人角色，可以使用各种武器和技能，灵...
2145,荣耀游戏是什么？,荣耀是《全职高手》中虚构的一款大型多人在线角色扮演游戏（MMORPG），是职业选手们进行竞技...
2146,嘉世战队在小说中是什么样的存在？,嘉世战队是《全职高手》中最初的三冠王战队，叶修曾是这支战队的队长，但由于管理层的决定，叶修被...


# 处理数据集

In [7]:
tokenizer = AutoTokenizer.from_pretrained(mode_path, use_fast=False,trust_remote_code=True)
tokenizer

Qwen2Tokenizer(name_or_path='/root/share/new_models/qwen/Qwen2-7B-Instruct', vocab_size=151643, model_max_length=131072, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|im_end|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>']}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [8]:
def process_func(example):
    MAX_LENGTH = 1024   # Llama分词器会将一个中文字切分为多个token，因此需要放开一些最大长度，保证数据的完整性
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer(f'''<|im_start|>system\n### Context:
现在你是最了解小说《全职高手》的人。你是读者的朋友，愿意分享见闻，解答读者关于《全职高手》或更广泛话题的好奇。<|im_end|>\n<|im_start|>user\n### 提问:{example['question']}<|im_end|>\n<|im_start|>assistant\n''', add_special_tokens=False)  # add_special_tokens 不在开头加 special_tokens
    response = tokenizer(f"{example['answer']}", add_special_tokens=False)
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]  # 因为eos token咱们也是要关注的所以 补充为1
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]  
    if len(input_ids) > MAX_LENGTH:  # 做一个截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [9]:
tokenized_id = ds.map(process_func, remove_columns=ds.column_names)
tokenized_id

Map:   0%|          | 0/2148 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 2148
})

In [10]:
tokenizer.decode(tokenized_id[0]['input_ids'])

'<|im_start|>system\n### Context:\n现在你是最了解小说《全职高手》的人。你是读者的朋友，愿意分享见闻，解答读者关于《全职高手》或更广泛话题的好奇。<|im_end|>\n<|im_start|>user\n### 提问:叶秋在被嘉世俱乐部驱逐后，如何重新找回自己的巅峰之路？<|im_end|>\n<|im_start|>assistant\n叶秋在被嘉世俱乐部驱逐后，通过在网游荣耀中创建账号‘一叶之秋’，并重新投入游戏，带着对往昔的回忆和自制的千机伞武器，开始了重返巅峰之路。他拒绝了对手的邀战，从嘉世俱乐部退出，前往网吧成为网管，以此为起点，借助对职业游戏的深厚理解与经验，他重新找回了自己作为顶尖高手的地位，通过不断练习与竞技，最终再次成为荣耀职业联盟中备受瞩目的‘斗神’角色。<|endoftext|>'

In [11]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_id[1]["labels"])))

'叶修在离开俱乐部时没有太多的话别，他一共只说了八个字：‘休息一年，然后回来。’苏沐橙站在俱乐部的大门口，她看着叶秋一路走到消失，叶秋不住地回身朝自己挥着手，苏沐橙早已经泪流满面。<|endoftext|>'

# 创建模型

In [12]:
import torch

model = AutoModelForCausalLM.from_pretrained(mode_path, device_map="auto",torch_dtype=torch.bfloat16,trust_remote_code=True)
model

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear(in_features=3584, out_features=3584, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm()
        (post_attention_layernorm): Qwen2RMSNorm()
      )
    )
    (norm): Qwen2RMSNorm()
  )
  (lm_head): Lin

In [13]:
model.enable_input_require_grads() # 开启梯度检查点时，要执行该方法

In [14]:
model.dtype

torch.bfloat16

# lora 

In [15]:
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, 
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    # target_modules=["ffn_proj", "gate_proj"],
    inference_mode=False, # 训练模式
    r=8, # Lora 秩
    lora_alpha=32, # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1# Dropout 比例
)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'down_proj', 'q_proj', 'v_proj', 'k_proj', 'o_proj', 'gate_proj', 'up_proj'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [16]:
model = get_peft_model(model, config)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='/root/share/new_models/qwen/Qwen2-7B-Instruct', revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'down_proj', 'q_proj', 'v_proj', 'k_proj', 'o_proj', 'gate_proj', 'up_proj'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [17]:
model.print_trainable_parameters()

trainable params: 20,185,088 || all params: 7,635,801,600 || trainable%: 0.26434798934534914


# 配置训练参数

In [18]:
args = TrainingArguments(
    output_dir=output_lora_dir,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    logging_steps=10,
    num_train_epochs=10,
    save_steps=100, # 为了快速演示，这里设置10，建议你设置成100
    learning_rate=1e-4,
    save_on_each_node=True,
    gradient_checkpointing=True
)

In [19]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_id,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

In [20]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/root/.conda/envs/logic/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/root/.conda/envs/logic/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss
10,2.522800
20,2.426300
30,2.326900
40,2.302500
50,2.281700
60,2.264600
70,2.279400
80,2.234400
90,2.301000
100,2.264000


/root/.conda/envs/logic/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in /root/share/new_models/qwen/Qwen2-7B-Instruct - will assume that the vocabulary was not modified.
  warnings.warn(
/root/.conda/envs/logic/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/root/.conda/envs/logic/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.

TrainOutput(global_step=1340, training_loss=1.39375392750128, metrics={'train_runtime': 7082.6514, 'train_samples_per_second': 3.033, 'train_steps_per_second': 0.189, 'total_flos': 2.3478736260549427e+17, 'train_loss': 1.39375392750128, 'epoch': 9.981378026070763})

# 合并加载模型（可选）

In [13]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel

# mode_path = '/share/new_models/qwen/Qwen2-7B-Instruct/'
# lora_path = './output/Qwen2_instruct_lora/checkpoint-2600' # 这里改称你的 lora 输出对应 checkpoint 地址

# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(mode_path, trust_remote_code=True)

# 加载模型
model = AutoModelForCausalLM.from_pretrained(mode_path, device_map="auto",torch_dtype=torch.bfloat16, trust_remote_code=True).eval()

# 加载lora权重
# model = PeftModel.from_pretrained(model, model_id=lora_path)
model = model.merge_and_unload() 
model.save_pretrained("./merge")

tokenizer.save_pretrained("./merge", max_shard_size="2048MB", safe_serialization=True)





# # 模型合并存储

# new_model_directory = "./merged_model"
# merged_model = model.merge_and_unload()
# # 将权重保存为safetensors格式的权重, 且每个权重文件最大不超过2GB(2048MB)
# merged_model.save_pretrained(new_model_directory, max_shard_size="2048MB", safe_serialization=True)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

('./merge/tokenizer_config.json',
 './merge/special_tokens_map.json',
 './merge/vocab.json',
 './merge/merges.txt',
 './merge/added_tokens.json',
 './merge/tokenizer.json')

# 验证一下（可选）

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel

# mode_path = '/share/new_models/qwen/Qwen2-7B-Instruct/'
# lora_path = './output/Qwen2_instruct_lora/checkpoint-200' # 这里改称你的 lora 输出对应 checkpoint 地址
# mode_path = './merge/'
# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(mode_path, trust_remote_code=True)

# 加载模型
model = AutoModelForCausalLM.from_pretrained(mode_path, device_map="auto",torch_dtype=torch.bfloat16, trust_remote_code=True).eval()

# 加载lora权重
model = PeftModel.from_pretrained(model, model_id=lora_path)

# prompt = "题目如下：有一个列表，找出该列表的最后一个元素。\n\n下列选项中哪个是列表 `[a, b, c, d]` 的最后一个元素？A: a; B: b; C: c; D: d"
while True:
    question = input("请输入问题>>\n")
    prompt = f"""### Context:
    现在你是最了解小说《全职高手》的人。你是读者的朋友，愿意分享见闻，解答读者关于《全职高手》或更广泛话题的好奇。问题如下：{question}
    """
           
    inputs = tokenizer.apply_chat_template([{"role": "user", "content": prompt}],
                                           add_generation_prompt=True,
                                           tokenize=True,
                                           return_tensors="pt",
                                           return_dict=True
                                           ).to('cuda')
    
    
    gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1}
    with torch.no_grad():
        outputs = model.generate(**inputs, **gen_kwargs)
        outputs = outputs[:, inputs['input_ids'].shape[1]:]
        print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/root/.conda/envs/logic/lib/python3.10/site-packages/accelerate/utils/modeling.py:1405: UserWarning: Current model requires 268437504 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device device because they were offloaded to the cpu.


请输入问题>>
 天机伞能干什么


天机伞是《全职高手》中角色叶修所使用的自制武器，具有多种变化形态和技能。它能变形为伞、矛、叉、刀等多种武器形态，提供不同的战斗效果和策略。此外，天机伞还具有隐身和加速移动等特殊能力，使用户在战斗中获得优势。在游戏《荣耀》中，天机伞作为叶修的重要装备，对他的战斗风格和战术有着关键影响。


...

请输入问题>>
 罗辑玩的什么角色


罗辑玩的是元素法师角色


请输入问题>>
 叫什么


叶修


KeyboardInterrupt: Interrupted by user